In [22]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
import numpy as np
from Utils.utils import create_URM, create_ICM, combine_matrices, create_submission, write_submission, load_users_for_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Evaluation.K_FoldEvaluator import K_FoldEvaluator
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeTwoModelsByScores, MergeThreeModelsByScores

In [39]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()
URM_stacked = combine_matrices(ICM=ICM, URM=URM)

In [40]:
URM

<13650x18059 sparse matrix of type '<class 'numpy.float64'>'
	with 5285664 stored elements in Compressed Sparse Row format>

In [69]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13634 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13631 ( 0.1%) Users that have less than 1 test interactions


In [70]:
stacked_train = combine_matrices(ICM=ICM, URM= URM_train)

In [11]:
### RP3Beta Recommender
RP3Beta_recommender = RP3betaRecommender(URM_train=URM_train)
RP3Beta_recommender.fit(topK=181, alpha=0.5709402717259106, beta=0.3566066452521787, implicit=True, normalize_similarity=True)

RP3betaRecommender: URM Detected 2 ( 0.0%) users with no interactions.


In [12]:
##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_stacked)
SLIME_recommender.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)


 20%|███████▌                              | 3592/18059 [02:40<15:27, 15.59it/s]


 39%|██████████████▊                       | 7016/18059 [05:09<12:09, 15.14it/s]


 58%|█████████████████████▌               | 10536/18059 [07:46<08:33, 14.65it/s]


 77%|████████████████████████████▎        | 13832/18059 [10:02<02:54, 24.18it/s]


 95%|███████████████████████████████████  | 17128/18059 [12:19<00:28, 33.16it/s]


100%|████████████████████████████████████▉| 18032/18059 [12:56<00:00, 45.41it/s]

In [13]:
similarity_slime = SLIME_recommender.W_sparse
similarity_rp3 = RP3Beta_recommender.W_sparse

In [14]:

similarity_hybrid = ItemKNNSimilarityHybridRecommender(URM_train = URM_stacked,
                                                      Similarity_1 = similarity_rp3,
                                                      Similarity_2 = similarity_slime)
similarity_hybrid.fit(topK=494, alpha=0.1597863887003789 )

In [15]:
## IALS Recommender
IALS_recommender = IALSRecommender(URM_train=URM_stacked)
IALS_recommender.fit(epochs=10, num_factors=42, alpha=0.7617528864750021, reg=8.926401306541349)

IALSRecommenderLinear: Epoch 1 of 10. Elapsed time 12.12 sec



100%|████████████████████████████████████▉| 18056/18059 [13:12<00:00, 45.41it/s]

IALSRecommenderLinear: Epoch 2 of 10. Elapsed time 23.26 sec
IALSRecommenderLinear: Epoch 3 of 10. Elapsed time 32.10 sec
IALSRecommenderLinear: Epoch 4 of 10. Elapsed time 41.04 sec
IALSRecommenderLinear: Epoch 5 of 10. Elapsed time 54.07 sec
IALSRecommenderLinear: Epoch 6 of 10. Elapsed time 1.08 min
IALSRecommenderLinear: Epoch 7 of 10. Elapsed time 1.25 min
IALSRecommenderLinear: Epoch 8 of 10. Elapsed time 1.42 min
IALSRecommenderLinear: Epoch 9 of 10. Elapsed time 1.59 min
IALSRecommenderLinear: Epoch 10 of 10. Elapsed time 1.77 min
IALSRecommenderLinear: Terminating at epoch 10. Elapsed time 1.77 min


In [18]:
##ItemKNNCF
ItemKNNCF_recommender = ItemKNNCFRecommender(URM_train = stacked_train)
ItemKNNCF_recommender.fit(topK=273, shrink=5.0, similarity="cosine")

ItemKNNCFRecommender: URM Detected 2 ( 0.0%) users with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 18059 (100.0%), 1018.21 column/sec. Elapsed time 17.74 sec


In [87]:
URM_dense = np.zeros((13650,18059),dtype="float64")
users_to_recommend = load_users_for_submission()

In [88]:
for i in range(0,len(users_to_recommend)):
    URM_dense[i]= ItemKNNCF_recommender._compute_item_score(user_id_array=i)

In [89]:
import scipy.sparse as sps
URM_dense = sps.coo_matrix(URM_dense)

In [90]:
URM_dense

<13650x18059 sparse matrix of type '<class 'numpy.float64'>'
	with 118100006 stored elements in COOrdinate format>

In [91]:
recommenders = [similarity_hybrid, IALS_recommender]
hybrid = MergeTwoModelsByScores(URM_dense, recommenders)
hybrid.fit(alpha=0.4555360263375717,beta=0.13654581079541908)

In [92]:
result_df, _ = evaluator_test.evaluateRecommender(hybrid)
print(result_df.loc[10])

EvaluatorHoldout: Processed 13634 (100.0%) in 1.11 min. Users per second: 204
PRECISION                      0.002391
PRECISION_RECALL_MIN_DEN       0.002508
RECALL                         0.000648
MAP                            0.001094
MAP_MIN_DEN                    0.001123
MRR                            0.010709
NDCG                           0.003112
F1                             0.001019
HIT_RATE                       0.023471
ARHR_ALL_HITS                  0.010826
NOVELTY                        0.008123
AVERAGE_POPULARITY             0.367805
DIVERSITY_MEAN_INTER_LIST        0.9886
DIVERSITY_HERFINDAHL           0.998853
COVERAGE_ITEM                  0.500637
COVERAGE_ITEM_CORRECT          0.014453
COVERAGE_USER                  0.998828
COVERAGE_USER_CORRECT          0.023443
DIVERSITY_GINI                 0.153464
SHANNON_ENTROPY               11.487628
RATIO_DIVERSITY_HERFINDAHL     0.998935
RATIO_DIVERSITY_GINI            0.24628
RATIO_SHANNON_ENTROPY          0.832546
RA